In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/Data/docnli")
os.getcwd()

'/content/drive/MyDrive/Data/docnli'

In [ ]:
!#pip install spacy spacy-experimental
!python -m spacy download en_core_web_lg

In [ ]:
!pip install https://github.com/explosion/spacy-experimental/releases/download/v0.6.1/en_coreference_web_trf-3.4.0a2-py3-none-any.whl

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")
# nlp_coref = spacy.load("en_coreference_web_trf")



In [ ]:
doc = nlp("The cats were startled by the dog as it growled at them. They ran away and hid behind the sofa, since the dog could not go there.")
for span in doc.spans :
    for t in doc.spans[span] :
        print((t.ent_id))
        print(doc.index(t))

In [ ]:
dir(token)

In [ ]:
for token in doc :
    # print(dir(token))
    print(token, token.idx)

In [ ]:
import pickle
import json
from transformers import LongformerModel, AutoTokenizer

In [ ]:
model = LongformerModel.from_pretrained("allenai/longformer-base-4096")
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

model = model.to("cuda")

In [ ]:
train_data_path = '/content/drive/MyDrive/Data/docnli/data/docnli/train_5sent_50ksample.json'
dev_data_path = '/content/drive/MyDrive/Data/docnli/data/docnli/dev_5sent_10ksample.json'

train_data_output_dir = '/content/drive/MyDrive/Data/docnli/data/longformer/train'
dev_data_output_dir = '/content/drive/MyDrive/Data/docnli/data/longformer/dev'

batch_size = 2

if not os.path.exists(train_data_output_dir) :
    os.mkdir(train_data_output_dir)

if not os.path.exists(dev_data_output_dir) :
    os.mkdir(dev_data_output_dir)

In [ ]:
train_data = json.load(open(train_data_path))
dev_data = json.load(open(dev_data_path))

In [ ]:
def batch(iterable , batch_size) :
    l = len(iterable)
    for i in range(0, l, batch_size) :
        yield iterable[i:min(l, i+batch_size)]

def save_dict_as_pickle(d , save_path) :
    with open(save_path, 'wb') as f :
        pickle.dump(d, f)


In [ ]:
start_index = int(len(os.listdir(train_data_output_dir))/batch_size)
train_batches = [train_batch for train_batch in batch(train_data, batch_size)][start_index : ]

In [ ]:
for data in batch(train_data, batch_size) :

    premise_hypothesis_pairs = [(d['premise'] , d['hypothesis']) for d in data]

    tokens = tokenizer(premise_hypothesis_pairs, padding='max_length' , truncation=True, return_tensors='pt')

    input_ids = tokens.input_ids.to("cuda")
    attention_mask = tokens.attention_mask.to("cuda")

    longformer_repr = model(input_ids=input_ids, attention_mask=attention_mask)
    longformer_repr = longformer_repr.last_hidden_state[:,0,:].cpu().detach().numpy()

    for i , d in enumerate(data) :
        d['longformer_repr'] = longformer_repr[i]
        save_path = os.path.join(train_data_output_dir, f"{d['id']}.pkl")
        save_dict_as_pickle(d , save_path)

    break

In [ ]:
tokens = tokenizer(premise_hypothesis_pairs, padding='max_length' , truncation=True, return_tensors='pt')

In [ ]:
input_ids = tokens.input_ids.to("cuda")
attention_mask = tokens.attention_mask.to("cuda")

In [ ]:
longformer_repr = model(input_ids=input_ids, attention_mask=attention_mask)

In [ ]:
longformer_repr.last_hidden_state[:,0,:]

In [ ]:
longformer_repr.last_hidden_state[:,0,:].cpu().detach().numpy()

In [ ]:
import glob
for f in glob.glob(train_data_output_dir + '/*') :

    print(pickle.load(open(f , 'rb'))['longformer_repr'].shape)

In [ ]:
for i in batch(list(range(12)) , 3) :
    print(i)

In [ ]:
12/3

## BERT Stuff

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import pickle
import json
import numpy as np
from nltk.tokenize import sent_tokenize
from tqdm.notebook import tqdm
from transformers import BertModel, AutoTokenizer

In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
'''
Variable Declaration
'''

train_data_path = 'data/docnli/train_5sent_50ksample.json'
dev_data_path = 'data/docnli/dev_5sent_10ksample.json'

save_dir = 'data/bert'
train_data_output_dir = 'data/bert/train'
dev_data_output_dir = 'data/bert/dev'
batch_size = 2

model_name = 'bert-base-cased'
tokenizer_name = 'bert-base-cased'

In [6]:
'''
Folder Creation
'''

if not os.path.exists(save_dir) :
    os.mkdir(save_dir)

if not os.path.exists(train_data_output_dir) :
    os.mkdir(train_data_output_dir)

if not os.path.exists(dev_data_output_dir) :
    os.mkdir(dev_data_output_dir)



In [7]:
'''
Load Models
'''

model = BertModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

model = model.to("cuda")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [37]:
'''
Function Definition
'''

def batch(iterable , batch_size) :
    l = len(iterable)
    for i in range(0, l, batch_size) :
        yield iterable[i:min(l, i+batch_size)]

def save_dict_as_pickle(d , save_path) :
    with open(save_path, 'wb') as f :
        pickle.dump(d, f)

def get_cls(text) :

    tokens = tokenizer(text, padding='max_length' , truncation=True, return_tensors='pt')

    input_ids = tokens.input_ids.to("cuda")
    attention_mask = tokens.attention_mask.to("cuda")

    cls = model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:,0,:]
    cls = cls.cpu().detach().numpy()

    return cls

def get_sentence_lvl_repr(text, max_num_sentences=5 , model_dim=768) :

    sentence_lvl_repr = np.zeros((max_num_sentences, model_dim))

    sent_tokenized_text = sent_tokenize(text)
    cls = get_cls(sent_tokenized_text)
    sentence_lvl_repr[:cls.shape[0]] = cls

    return sentence_lvl_repr

In [12]:
'''
Read data
'''

train_data = json.load(open(train_data_path))
dev_data = json.load(open(dev_data_path))

In [40]:
for data in train_data :

    premise = data['premise']
    premise_repr = get_sentence_lvl_repr(premise)

    hypothesis = data['hypothesis']
    hypothesis_repr = get_sentence_lvl_repr(hypothesis)

    data['premise_repr'] = premise_repr
    data['hypothesis_repr'] = hypothesis_repr

    save_path = os.path.join(train_data_output_dir, f"{data['id']}.pkl")
    save_dict_as_pickle(data , save_path)

    break

In [36]:
for data in dev_data :

    premise = data['premise']
    premise_repr = get_sentence_lvl_repr(premise)


    hypothesis = data['hypothesis']
    hypothesis_repr = get_sentence_lvl_repr(hypothesis)

    data['premise_repr'] = premise_repr
    data['hypothesis_repr'] = hypothesis_repr

    save_path = os.path.join(dev_data_output_dir, f"{data['id']}.pkl")
    save_dict_as_pickle(data , save_path)

    break

['Mappings is a first book of poems by Vikram Seth originally published by the Writers Workshop, Calcutta (now Kolkata), as a hand-set, hand-printed and hand-bound volume ("in Hardback or Flexiback") in 1980 or 1981 (the Flexiback edition copyright date is 1981).', "With the growth of Seth's reputation, the volume has been reprinted by mainstream publishers (ISBN\xa0 )."]
[[ 0.42323142 -0.14759561 -0.12737736 ... -0.04971308  0.05681191
   0.100229  ]
 [ 0.4572084  -0.09052125 -0.08401992 ... -0.2516827   0.26481402
   0.08174052]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
